In [ ]:
from ipynb.fs.full.procc import *
from statsmodels.nonparametric.smoothers_lowess import lowess
import seaborn as sns
from scipy.stats import sem, ttest_ind
from windrose import WindroseAxes
import matplotlib.cm as cm
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import sys
from windrose import WindroseAxes, WindAxes
#from metpy import calc
#from metpy.units import units
from scipy.signal import savgol_filter
import csv
import glob
from IPython.display import clear_output
from matplotlib.patches import Polygon

In [ ]:
# periapsis.....................01-15 - day 15
# vernal equinox................03-46 - day 148 = 0 °
# summer solstice...............06-51 - day 325 = 90 °
# apsis.........................07-18 - day 352
# autumnal equinox..............09-52 - day 508 = 180 °
# winter solstice...............12-46 - day 668 = 270 °

In [ ]:
day_lst = []
start = 147
for i in range(675):
    start+=1
    day_lst.append(start % 675)
    


In [ ]:
timestepSL = np.linspace(0,360,517050)
dailySL = np.linspace(0,360,675)

In [ ]:
ve_eqDaily = dailySL[0]
au_eqDaily = dailySL[360]

ve_eqTS = timestepSL[0]
au_eqTS = timestepSL[360*766]

rollbackDay = (148-1)*-1
rollbackTS = ((148*766)-1)*-1

In [ ]:
def ppercent(count, total):

    perc=float(count/total)*100

    text = "Percent completed: {:.2f}% ".format(perc)

    print(text)

    clear_output(wait=True)

In [ ]:
def calcDP(WAll, DirAll, UF):
    
    UT = UF * 24.3
    
    allDP = np.array([])
    allDP_STDERR = np.array([])

    for i in range(0,len(WAll)):

        W = WAll[i]
        Dir = DirAll[i]

        bins=[]
        windbins = np.arange(0,361,22.5)
        deg_midP = [(windbins[i]+windbins[i+1])/2 for i in range(0,len(windbins)-1)]

        for i in range(0,len(windbins)-1):
            if i == len(windbins)-1:
                bins.append((xr.where((Dir>=windbins[i]) & (Dir<=windbins[i+1]), True, False)))
            else:
                bins.append((xr.where((Dir>=windbins[i]) & (Dir<windbins[i+1]), True, False)))


        t = xr.where(W>UT,1,0)
        total=len(t)
        cum_sum = sum(t)
        t = float(cum_sum/total)

        bins = [(np.ma.masked_equal((bins[i]*W),0)).compressed() for i in range(0,len(bins))]

        binnedWavg = [np.mean(bins[i], axis=0) for i in range(0,len(bins))]

        binnedDP = [(binnedWavg[i]**2 * (binnedWavg[i] - UT) * t) / 100. for i in range(0,len(bins))]

        C = [binnedDP[i]*np.sin(np.deg2rad(deg_midP[i])) for i in range(0,len(binnedDP))]
        D = [binnedDP[i]*np.cos(np.deg2rad(deg_midP[i])) for i in range(0,len(binnedDP))]

        C = np.nansum(C)
        D = np.nansum(D)

        if np.isnan(np.nansum(binnedDP)):
            DP = 0.
        else:
            DP = np.nansum(binnedDP)

        DP_STDERR = sem(binnedDP,nan_policy='omit')
        DP_STD = np.nanstd(binnedDP)

        allDP = np.append(DP, allDP)
        allDP_STDERR = np.append(allDP_STDERR, DP_STDERR)

        
        fill = np.nanmean(allDP_STDERR)
        allDP_STDERR = np.ma.masked_invalid(allDP_STDERR)
        allDP_STDERR = np.ma.filled(allDP_STDERR,fill)

        #clear_output()
        
    return allDP, allDP_STDERR

In [ ]:
# count = 0
# total = len(runs) * len(regions) * len(locations) * len(UT)

def dpArea(run, region='', location=''):
    
    dp_dict = {}
    
    if region == 'Equator':
        location = ""
        timestep = "Timestep"
    else:
        timestep = ""

    ## Recalculating DP since issue with artifact in timeseries


    W_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYYrAvgTSt_W.csv')[0]
    DIR_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYYrAvgTSt_Dir.csv')[0]
    
    W = openCSV(W_path)
    DIR = openCSV(DIR_path)
    
    allDP, allDP_STDERR = calcDP(W, DIR, 0.005)
    
    dp_dict['DP'] = allDP
    dp_dict['DPSTDERR'] = allDP_STDERR 
    
    return dp_dict


In [ ]:
def calcUSTAR(run, region='', location='', z0 = 0.0246, z = 10, k = 0.4):
    
    if region == 'Equator':
        location = ""
        timestep = "Timestep"
    else:
        timestep = ""

    W_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYYrAvgTSt_W.csv')[0]
    
    W = openCSV(W_path)
    
    W = np.array(flattenArray(W))

    coeff = (1/np.log(z/z0))*k

    u_star = coeff*W
    
    return u_star

In [ ]:
def getData(run, region, filename, location = "", flatten = False):
    
    if region == 'Equator':
        location = ""
        timestep = "Timestep"
    else:
        timestep = ""
    
    path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'\
                     +run+'/'+region+'/'+timestep+location+'/*'+filename+'.csv')[0]
    
    dataset = openCSV(path)
    
    if flatten:
        return flattenArray(dataset)
    else:
        return dataset

In [ ]:
def makeDict(dict, run ,region='', locations='', DP=False, USTAR=False, fn=''):
    
    if DP:
        if not locations:
            dict[run] = dpArea(run, region)
        else:
            loc_dict = {}
            for location in locations:
                loc_dict[location] = dpArea(run, region, location)

            dict[run] = loc_dict
    
    elif USTAR:
        if not locations:
            dict[run] = calcUSTAR(run, region)
        else:
            loc_dict = {}
            for location in locations:
                loc_dict[location] = calcUSTAR(run, region, location)

            dict[run] = loc_dict
    
    else:
        if not locations:
            dict[run] = getData(run, region, fn, flatten=True)
        else:
            loc_dict = {}
            for location in locations:
                loc_dict[location] = getData(run, region, fn, location = location, flatten=True)
            dict[run] = loc_dict
                

In [ ]:
def makeBoxPlot(dict, runs, location, xlabel, ylabel, bottom, top, show=False, 
                title = '', var = '', N=675, output = '', yticks = np.array([])):

    plt.clf()
    

    if var:
        if location == 'Equator':
            box1 = dict[runs[0]][var]
            box2 = dict[runs[1]][var]
            box3 = dict[runs[2]][var]
            box4 = dict[runs[3]][var]
        else:
            box1 = dict[runs[0]][location][var]
            box2 = dict[runs[1]][location][var]
            box3 = dict[runs[2]][location][var]
            box4 = dict[runs[3]][location][var]
    elif location == 'Equator':
        box1 = dict[runs[0]]
        box2 = dict[runs[1]]
        box3 = dict[runs[2]]
        box4 = dict[runs[3]]
    else:
        box1 = dict[runs[0]][location]
        box2 = dict[runs[1]][location]
        box3 = dict[runs[2]][location]
        box4 = dict[runs[3]][location]

    # Generate some random indices that we'll use to resample the original data
    # arrays. For code brevity, just use the same random indices for each array

    data = [
        box1,
        box2,
        box3,
        box4
        ]
    
#     color_dict =  {'patch_artist': True,
#              'capprops': dict(color='k'),
#              'medianprops': dict(color='k')}
    
    fig, ax1 = plt.subplots(figsize=(10, 6))
    #fig.canvas.manager.set_window_title(title)
    fig.subplots_adjust(left=0.075, right=0.95, top=0.9, bottom=0.25)

    bp = ax1.boxplot(data, notch=0, sym='+', vert=1, whis=1.5)
    plt.setp(bp['boxes'], color='dimgrey')
    plt.setp(bp['whiskers'], color='dimgrey')
    plt.setp(bp['fliers'], color='dimgrey', marker='+')
    plt.setp(bp['medians'], color='dimgrey')
    plt.setp(bp['caps'], color='dimgrey')

    # Add a horizontal grid to the plot, but make it very light in color
    # so we can use it for reading data values but not be distracting
    ax1.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',
                   alpha=0.5)

    if title:
        ax1.set(
            title=title
        )
        
    ax1.set(
        axisbelow=True,  # Hide the grid behind plot objects
        xlabel=xlabel,
        ylabel=ylabel,
    )

    # Now fill the boxes with desired colors
    num_boxes = len(data)
    box_colors = ['skyblue']*num_boxes
    medians = np.empty(num_boxes)
    for i in range(num_boxes):
        box = bp['boxes'][i]
        box_x = []
        box_y = []
        for j in range(5):
            box_x.append(box.get_xdata()[j])
            box_y.append(box.get_ydata()[j])
        box_coords = np.column_stack([box_x, box_y])
        # Alternate between Dark Khaki and Royal Blue
        
        ax1.add_patch(Polygon(box_coords, facecolor=box_colors[i]))
        
        # Now draw the median lines back over what we just filled in

        med = bp['medians'][i]
        median_x = []
        median_y = []
        for j in range(2):
            median_x.append(med.get_xdata()[j])
            median_y.append(med.get_ydata()[j])
            #ax1.plot(median_x, median_y, color = 'black',lw=2)
        medians[i] = median_y[0]
        
        # Finally, overplot the sample averages, with horizontal alignment
        # in the center of each box
        ax1.plot(np.average(med.get_xdata()), np.average(data[i]),
                 color='w', marker='X', markeredgecolor='k')

    # Set the axes ranges and axes labels
    ax1.set_xlim(0.5, num_boxes + 0.5)
    top = top
    bottom = bottom
    ax1.set_ylim(bottom, top)
    ax1.set_xticklabels(runs,
                        rotation=45, fontsize=12)
    if yticks.any():
        ax1.set_yticks(yticks)

    # Due to the Y-axis scale being different across samples, it can be
    # hard to compare differences in medians across the samples. Add upper
    # X-axis tick labels with the sample medians to aid in comparison
    # (just use two decimal places of precision)
    pos = np.arange(num_boxes) + 1
    upper_labels = [str(round(s, 4)) for s in medians]
    weights = ['bold', 'semibold']
    for tick, label in zip(range(num_boxes), ax1.get_xticklabels()):
        k = tick % 2
        ax1.text(pos[tick], .95, upper_labels[tick],
                 transform=ax1.get_xaxis_transform(),
                 horizontalalignment='center', size='x-small',
                 weight=weights[k], color='black')

    # Finally, add a basic legend
    fig.text(0.12, 0.2, 'x', color='white', backgroundcolor='dimgray',
             weight='roman', size='small')
    
    fig.text(0.02, 0.2, ' Average Value', color='black', weight='roman',
             size='x-small')

    if output:
        plt.savefig(output+'_boxp.png')
    
    if show:
        plt.show()
    else: 
        plt.clf()
        


In [ ]:
#https://en.wikipedia.org/wiki/Savitzky%E2%80%93Golay_filter

def compSeries(data1, data2, labels1, labels2, xlabel, ylabel, err1 = np.array([]), err2 = np.array([]), color1 = 'tab:blue', color2 = 'tab:orange', 
                   xticks = np.arange(0,361,30), yticks = np.array([]), ylim = np.array([]), output = ''):
    
    plt.clf()
    
    x = dailySL
    
    yhat1 = savgol_filter(data1, 51, 3) # window size 51, polynomial order 3
    yhat2 = savgol_filter(data2, 51, 3)
    
    if err1.any() and err2.any():
        order = [0,2,1,3]
        
        data1_upper = data1 + err1
        data1_lower = data1 - err1

        data2_upper = data2 + err2
        data2_lower = data2 - err2

        plt.fill_between(x,np.roll(data1_upper,-147),np.roll(data1_lower,-147),alpha=0.2, color = color1, label = labels1[1])
        plt.fill_between(x,np.roll(data2_upper,-147),np.roll(data2_lower,-147),alpha=0.2, color = color2, label = labels2[1])



    else: 
        order = [2,0,3,1]
        
        plt.plot(dailySL, np.roll(data1,-147), lw = 1, color = color1, label = labels1[1], alpha = 0.2)
        plt.plot(dailySL, np.roll(data2,-147), lw = 1, color = color2, label = labels2[1], alpha = 0.2)
        
    plt.plot(dailySL, np.roll(yhat1,-147), lw = 1.5, color = color1, label = labels1[0])
    plt.plot(dailySL, np.roll(yhat2,-147), lw = 1.5, color = color2, label = labels2[0])
    
    plt.xlabel(xlabel)
    plt.xticks(xticks)
    
    plt.ylabel(ylabel)
    
    if yticks.any():
        plt.yticks(yticks)
    if ylim.any():
        plt.ylim(ylim)
    
    handles, labels = plt.gca().get_legend_handles_labels()

    plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order]) 
    
    plt.axvspan(ve_eqDaily - 10, ve_eqDaily + 10, color='orange', alpha=0.4)
    plt.axvspan(au_eqDaily - 10, au_eqDaily + 10, color='orange', alpha=0.4)
    
    if output:
        plt.savefig(output+'compTS.png', bbox_inches='tight')


In [ ]:
def histPlot(VAR, xlabel = '', ylabel = 'Frequency', output = '', binwidth=[], bins=[]):
    
    plt.clf()
    plt.figure(figsize=(8, 6), dpi=100)
    
    if binwidth:
    
        ax = sns.histplot(VAR,
                      binwidth=binwidth,
                      kde=True,
                      stat='density',
                      element = 'bars',
                      edgecolor = 'white')
        ax.set(xlabel=xlabel, ylabel=ylabel)
    
    elif bins:
        
        ax = sns.histplot(VAR,
                      bins=bins,
                      kde=True,
                      stat='density',
                      element = 'bars',
                      edgecolor = 'white')
        ax.set(xlabel=xlabel, ylabel=ylabel)
        
        
    else:
        
        ax = sns.histplot(VAR,
                      kde=True,
                      stat='density',
                      element = 'bars',
                      edgecolor = 'white')
        ax.set(xlabel=xlabel, ylabel=ylabel)
    
    if output:
        plt.savefig(output+'_HIST.png', bbox_inches='tight')
    


In [ ]:
def plotWindrose(DIR, W, output = '', anchor = (1., 1.), show=False, bins = np.arange(0,3.1,0.3)):

    plt.clf()
    ax = WindroseAxes.from_ax(theta_labels=['E','NE','N','NW','W','SW','S','SE'])
    ax.bar(DIR, W, bins = bins, opening=0.8, blowto = False, normed=True, edgecolor='white') 
    ax.set_legend(units='m s$^{-1}$',bbox_to_anchor = anchor)
    
    if output:
        plt.savefig(output+'WRose.png', bbox_inches='tight')
    
    if show:
        plt.show()
    else:
        plt.clf()
    

    
    #ax = WindAxes.from_ax()
    #ax, params = ax.pdf(W, bins=bins)
    #plt.show()

In [ ]:
def assignDays(data):
    
    data_daily = []
    
    for i in range(0,675):
        b = 766*i
        e = 766*(i+1)
        temp = data[b:e]
        data_daily.append(temp)
        
    return data_daily

In [ ]:
def seriesPlot(data, ylabel, timescale, yticks = np.array([]), ylim = np.array([]), \
               output = '', average = False, error=np.array([])):
    
    plt.clf()
    
    plt.figure(figsize=(12, 6), dpi=100)
    plt.rcParams.update({'font.size': 14})
    
    if timescale.lower() == 'daily':
        
        x = dailySL
        rb = rollbackDay
        
        plt.plot(x,np.roll(data,rb),linewidth=1,color='black')
        plt.axvspan(ve_eqDaily - 10, ve_eqDaily + 10, color='orange', alpha=0.4)
        plt.axvspan(au_eqDaily - 10, au_eqDaily + 10, color='orange', alpha=0.4)
        plt.xlabel('Solar Longitude (°)')
        plt.ylabel(ylabel)
        plt.xticks([0,90,180,270,360])

    elif timescale.lower() == 'timestep':
        
        x = timestepSL
        rb = rollbackTS
        
        if average:   
            
            data_daily = assignDays(data)
            x = dailySL
            rb = rollbackDay
            
            data = np.mean(data_daily,axis=1)
        
        plt.plot(x,np.roll(data,rb),linewidth=1,color='black')
        plt.axvspan(ve_eqTS - 10, ve_eqTS + 10, color='orange', alpha=0.4)
        plt.axvspan(au_eqTS - 10, au_eqTS + 10, color='orange', alpha=0.4)
        plt.xlabel('Solar Longitude (°)')
        plt.ylabel(ylabel)
        plt.xticks([0,90,180,270,360])
        
    if yticks.any():
        plt.yticks(yticks)
    if ylim.any():
        plt.ylim(ylim)
        
    if error.any():
        upper = data + error
        lower = data - error
        plt.fill_between(x,np.roll(upper,rb),np.roll(lower,rb),alpha=0.7,lw=2.5)
        
    if output:
        plt.savefig(output+'_TS.png', bbox_inches='tight')

In [ ]:
def countWesterlies(W, DIR, UT):
    
    wstly_ut_days = []
    wstly_days = []

    for i, day in enumerate(W):

        for j, wind in enumerate(day):

            deg = DIR[i][j] 

            if deg > 0. and deg < 180.:
                wstly_days.append(i)

                if wind>=UT:
                    wstly_ut_days.append(i)
                
    return wstly_ut_days, wstly_days

In [ ]:
def convDayToSL(arr):
    convArr = [dailySL[day_lst.index(day)] for day in arr]
    return convArr

In [ ]:
def flattenArray(arr): #flatten list of lists
    flat_arr = []
    
    for list in arr:
        for val in list:
            flat_arr.append(val)

    return flat_arr

In [ ]:
def openCSV(fn):
    array = []

    with open(fn, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            remake_row = []
            for num in row:
                remake_row.append(float(num))
            array.append(remake_row)

    return np.array(array)

In [ ]:
## count++ per wind per day
def plotWindFreq(W, DIR, UT, output = '', show = False):
    
    wstly_ut_days, wstly_days = countWesterlies(W, DIR, UT)
    
    SLdaysWUT = convDayToSL(wstly_ut_days)
    SLdaysW = convDayToSL(wstly_days)

    plt.clf()
    plt.style.use('seaborn-deep')

    bins=np.linspace(8.,360.,45)

    plt.hist([SLdaysW, SLdaysWUT], bins=15, label=['Westerly Winds Above Threshold','Total Westerly Winds'],
             histtype = 'stepfilled', color = ['tab:cyan', 'tab:blue'],
             weights=[np.ones(len(SLdaysW)) / len(SLdaysW), np.ones(len(SLdaysWUT)) / len(SLdaysWUT)])

    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    plt.xticks(np.arange(0.,361.,30))
    plt.xlabel('Solar Longitude (°)')
    plt.ylabel('Percent (%)')
    plt.legend(loc='upper right')
    
    if output:
        plt.savefig(output+'WFREQ.png', bbox_inches='tight')
        
    if show:
        plt.show()
    else:
        plt.clf()

In [ ]:
def zonalGusts(U):
    
    wmax = np.zeros(675)
    emax = np.zeros(675)

    #index for day too?

    for index, day in enumerate(U):

        westerly = []
        easterly = []

        for wind in day:
            if wind > 0.:
                westerly.append(wind)
            else: 
                easterly.append(wind)

        if westerly:
            wmax[index] = np.max(westerly)


        if easterly:
            emax[index] = np.max(easterly)
        
    wmax = np.ma.masked_where(wmax == 0., wmax)
    emax = np.ma.masked_where(emax == 0., emax)
    
    return wmax, emax
        

In [ ]:
def plotScatter(x1, x2, show=False, output = ''):
    
    plt.clf()

    plt.scatter(dailySL, np.roll(np.abs(x1),-147),marker='.',label='Max Easterly Zonal Wind', color='tab:orange')
    plt.scatter(dailySL, np.roll(x2,-147),marker='.',label='Max Westerly Zonal Wind', color='tab:blue')
    
    plt.xlabel('Solar Longitude (°)')
    plt.ylabel('Wind Speed (m/s)')
    
    plt.xticks(np.arange(0.,361.,30))

    plt.ylim([-0.05,0.6])
    plt.yticks(np.arange(0.,0.7,0.1))
    
    plt.legend(loc ='upper right')
    
    
    if output:
        plt.savefig(output+'SCTR.png', bbox_inches='tight')
        
    if show:
        plt.show()
    else: 
        plt.clf()

In [ ]:
runs = ['FL', 'WT', 'Z0', 'WTZ0']
regions = ['Belet', 'Equator', 'Fensal', 'ShangriLa']
locations = ['Center', 'EastEdge'] 

total = len(runs)*len(regions)*len(locations)

## Windroses

In [ ]:
count = 0

for run in runs:
    for region in regions:
        for location in locations:
            if region == 'Equator':
                location = ""
                timestep = "Timestep"
            else:
                timestep = ""
                
            name = "{}_{}_{}_".format(run,region,location)
            output = "/Users/maxcollins/Desktop/organized images/Windrose (right this time I swear)/"+ name

            W_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYYrAvgTSt_W.csv')[0]
            DIR_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYYrAvgTSt_Dir.csv')[0]
            
            W = openCSV(W_path)
            DIR = openCSV(DIR_path)
            
            W = flattenArray(W)
            DIR = flattenArray(DIR)
            
            print("Current: ", W_path)
            print("Current: ", DIR_path)
            
            plotWindrose(DIR, W, anchor = (-0.12,0.6), output = output)
            
            output = "/Users/maxcollins/Desktop/organized images/Histograms/"+ name + 'W'
            
            histPlot(W, xlabel = 'Wind Speed (m/s)', binwidth=0.025, output = output)
            
            output = "/Users/maxcollins/Desktop/organized images/Histograms/"+ name + 'DIR'
            
            histPlot(DIR, xlabel = 'Direction (°)', binwidth=5, output = output)
                
            count+=1

            perc=float(count/total)*100
            
            text = "Percent completed: {:.2f}% ".format(perc)

            print(text)

            clear_output(wait=True)
            

## Zonal Winds Histograms

In [ ]:
count = 0

for run in runs:
    for region in regions:
        for location in locations:
            if region == 'Equator':
                location = ""
                timestep = "Timestep"
            else:
                timestep = ""
                
            name = "{}_{}_{}_".format(run,region,location)

            U_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYYrAvgTSt_U.csv')[0]
            
            U = openCSV(U_path)
            
            U = flattenArray(U)
            
            westerly = [u for u in U if u>0.]
            easterly = [u for u in U if u<0.]
            
            easterly = np.abs(easterly)
            
            print("Current: ", U_path)
            
            output = "/Users/maxcollins/Desktop/organized images/Histograms/"+ name + 'WEST'
            
            histPlot(westerly, xlabel = 'Wind Speed (m/s)', binwidth=0.025, output = output)
            
            output = "/Users/maxcollins/Desktop/organized images/Histograms/"+ name + 'EAST'
            
            histPlot(easterly, xlabel = 'Wind Speed (m/s)', binwidth=0.025, output = output)
            
            output = "/Users/maxcollins/Desktop/organized images/Histograms/"+ name + 'ALL_U'
            
            histPlot(U, xlabel = 'Wind Speed (m/s)', binwidth=0.025, output = output)
                
            count+=1

            perc=float(count/total)*100
            
            text = "Percent completed: {:.2f}% ".format(perc)

            print(text)

            clear_output(wait=True)

## Drift Potential Windrose and Histograms

In [ ]:
UT = ['001', '003', '004', '005', '006']

count = 0
total = len(runs) * len(regions) * len(locations) * len(UT)

for run in runs:
    for region in regions:
        for location in locations:
            for ut in UT:
                if region == 'Equator':
                    location = ""
                    timestep = "Timestep"
                else:
                    timestep = ""

                name = "{}_{}_{}_".format(run,region,location)

                DP_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/BinnedDP/*'+ut+'_binDP.csv')[0]

                DP = openCSV(DP_path)

                DP = flattenArray(DP)

                DIR_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/BinnedDP/*'+ut+'_binDIR.csv')[0]

                DIR = openCSV(DIR_path)

                DIR = flattenArray(DIR)

                print("Current: ", DIR_path)
                print("Current: ", DP_path)
                
                output = "/Users/maxcollins/Desktop/organized images/Windrose (right this time I swear)/"+ name + '_' + ut + '_DP' 
                
                #plotWindrose(DIR, DP, anchor = (-0.12,0.6), output = output)

                output = "/Users/maxcollins/Desktop/organized images/Histograms/"+ name + 'DPDIR'

                histPlot(DIR, xlabel = 'Drift Potential Direction (°)', bins=16, output = output)
                
                output = "/Users/maxcollins/Desktop/organized images/Histograms/"+ name + 'DP'
                
                histPlot(DP, xlabel = 'Drift Potential (VU)', bins=16, output = output)

                count+=1

                perc=float(count/total)*100

                text = "Percent completed: {:.2f}% ".format(perc)

                print(text)

                clear_output(wait=True)
                
                
                            
                

## Timeseries DP with uncertainty and highlight equinoxes


In [ ]:
fensal_dictDP = {}
belet_dictDP = {}
shangrila_dictDP = {}
eq_dictDP = {}

region_dictsDP = [belet_dictDP, eq_dictDP, fensal_dictDP, shangrila_dictDP]

count = 0
total = len(runs) * len(regions)

for index, region in enumerate(regions):
    for run in runs:
        
        if region == 'Equator':
            locations=''
        else:
            locations = ['Center', 'EastEdge']
            
        makeDict(region_dictsDP[index], run, region, locations=locations, DP=True)
    
        count+=1
    
        ppercent(count,total)

#### Series plots for each seperately

In [ ]:
region_dictsDP = [belet_dictDP, eq_dictDP, fensal_dictDP, shangrila_dictDP]
region_names = ['Belet', 'Equator', 'Fensal', 'ShangriLa']
locations = ['Center', 'EastEdge']
runs = ['FL', 'WT', 'Z0', 'WTZ0']

In [ ]:
total = len(region_dict) * len(runs) * len(locations)
count=0

for index, region_dict in enumerate(region_dicts):
    for run in runs:
        for location in locations:           
            try:
                y =  region_dict[run][location]['DP']
                error = region_dict[run][location]['DPSTDERR']
                name = "{}_{}_{}_".format(run,region_names[index],location)
            except KeyError:
                y =  region_dict[run]['DP']
                error = region_dict[run]['DPSTDERR']
                name = "{}_{}_".format(run,region_names[index])
                
            output = "/Users/maxcollins/Desktop/organized images/Timeseries/"+ name + 'DP'
            if region_names[index] == 'Belet' and location == 'EastEdge':
                yticks = np.arange(0.,0.016,0.002)
                ylim = np.array([0.,0.015])
            elif region_names[index] == 'ShangriLa' and location == 'EastEdge':
                yticks = np.arange(0.,0.026,0.002)
                ylim = np.array([0.,0.025])
            else:
                yticks = np.arange(0.,0.013,0.002)
                ylim = np.array([0.,0.012])
                
            seriesPlot(y,'Drift Potential (VU)','daily', output = output, yticks = yticks, ylim = ylim, error = error)
            
            plt.clf()
            count+=1
            
            ppercent(count,total)
           

#### Comp series plots

In [ ]:
# region_dicts = [belet_dictDP, eq_dictDP, fensal_dictDP, shangrila_dictDP]
# region_names = ['Belet', 'Equator', 'Fensal', 'ShangriLa']
# locations = ['Center', 'EastEdge']
# runs = ['FL', 'WT', 'Z0', 'WTZ0']

comps = [('FL','WT'), ('FL','Z0'), ('WTZ0','Z0')]

In [ ]:
plt.clf()

count=0
total= len(region_dicts) * len(locations) * len(comps)

for index, region_dict in enumerate(region_dictsDP):
    for location in locations: 
        for comp in comps:
           
            if region_names[index] != 'Equator':
                
                name = "{}{}_{}_{}_DP".format(comp[0],comp[1],region_names[index],location)
                
                DP1 = region_dict[comp[0]][location]['DP']
                DP2 = region_dict[comp[1]][location]['DP']

                DP1_ERR = region_dict[comp[0]][location]['DPSTDERR']
                DP2_ERR = region_dict[comp[1]][location]['DPSTDERR']

            else:
                
                name = name = "{}{}_{}_DP".format(comp[0],comp[1],region_names[index])
                
                DP1 = region_dict[comp[0]]['DP']
                DP2 = region_dict[comp[1]]['DP']

                DP1_ERR = region_dict[comp[0]]['DPSTDERR']
                DP2_ERR = region_dict[comp[1]]['DPSTDERR']

            output = '/Users/maxcollins/Desktop/organized images/Timeseries/DP/Comparison/' + name

            compSeries(DP1, DP2, [comp[0], comp[0]+' Standard Error'], [comp[1],comp[1]+' Standard Error'], 
                       'Solar Longitude (°)', 'Drift Potential (VU)' , err1 = DP1_ERR, err2 = DP2_ERR, xticks = np.arange(0,361,30),
                       yticks = np.arange(0.,0.017,0.002), ylim = np.array([0.,0.017]), output = output)
            count+=1
            ppercent(count,total)



In [ ]:
plt.clf()

DP1 = belet_dictDP['FL']['EastEdge']['DP']
#DP2 = belet_dictDP['WT']['EastEdge']['DP']
DP3 = belet_dictDP['Z0']['EastEdge']['DP']
#DP4 = belet_dictDP['WTZ0']['EastEdge']['DP']

yhat1 = savgol_filter(DP1, 51, 3) # window size 51, polynomial order 3
#yhat2 = savgol_filter(DP2, 51, 3) # window size 51, polynomial order 3
yhat3 = savgol_filter(DP3, 51, 3) # window size 51, polynomial order 3
#yhat4 = savgol_filter(DP4, 51, 3) # window size 51, polynomial order 3

plt.plot(dailySL, np.roll(DP1,-147), lw = 1, alpha = 0.2,color = 'tab:orange')
#plt.plot(dailySL, np.roll(DP2,-147), lw = 1.5, alpha = 0.2, color = 'tab:blue' )
plt.plot(dailySL, np.roll(DP3,-147), lw = 1, alpha = 0.2,color = 'tab:purple')
#plt.plot(dailySL, np.roll(DP4,-147), lw = 1.5, alpha = 0.2, color = 'tab:red' )

plt.plot(x,np.roll(yhat1,-147),lw=1.2,color='tab:orange',label='FL')
#plt.plot(x,np.roll(yhat2,-147),lw=1,color='tab:blue',label='WT')
plt.plot(x,np.roll(yhat3,-147),lw=1.2,color='tab:purple',label='Z0')
#plt.plot(x,np.roll(yhat4,-147),'--',lw=1,color='tab:red',label='WTZ0')

plt.ylim([0.,0.016])
plt.yticks(np.arange(0.,0.017,0.002))
plt.xticks(np.arange(0,361,30))



plt.ylabel('Drift Potential (VU)')
plt.xlabel('Solar Longitude (°)')
plt.legend()
plt.show()

In [ ]:
plt.clf()

#DP1 = belet_dictDP['FL']['EastEdge']['DP']
DP2 = belet_dictDP['WT']['EastEdge']['DP']
#DP3 = belet_dictDP['Z0']['EastEdge']['DP']
DP4 = belet_dictDP['WTZ0']['EastEdge']['DP']

#yhat1 = savgol_filter(DP1, 51, 3) # window size 51, polynomial order 3
yhat2 = savgol_filter(DP2, 51, 3) # window size 51, polynomial order 3
#yhat3 = savgol_filter(DP3, 51, 3) # window size 51, polynomial order 3
yhat4 = savgol_filter(DP4, 51, 3) # window size 51, polynomial order 3

#plt.plot(dailySL, np.roll(DP1,-147), lw = 1, alpha = 0.2,color = 'tab:orange')
plt.plot(dailySL, np.roll(DP2,-147), lw = 1.5, alpha = 0.2, color = 'tab:blue' )
#plt.plot(dailySL, np.roll(DP3,-147), lw = 1, alpha = 0.2,color = 'tab:blue')
plt.plot(dailySL, np.roll(DP4,-147), lw = 1.5, alpha = 0.2, color = 'tab:red' )

#plt.plot(x,np.roll(yhat1,-147),lw=1.2,color='tab:orange',label='FL')
plt.plot(x,np.roll(yhat2,-147),lw=1,color='tab:blue',label='WT')
#plt.plot(x,np.roll(yhat3,-147),lw=1.2,color='tab:red',label='Z0')
plt.plot(x,np.roll(yhat4,-147),lw=1,color='tab:red',label='WTZ0')

plt.ylim([0.,0.016])
plt.yticks(np.arange(0.,0.017,0.002))
plt.xticks(np.arange(0,361,30))



plt.ylabel('Drift Potential (VU)')
plt.xlabel('Solar Longitude (°)')
plt.legend()
plt.show()

In [ ]:
plt.clf()
plt.plot(dailySL, np.roll(belet_dictDP['WTZ0']['Center']['DP'],-147), lw = 0.5, label = 'WTZ0')
plt.plot(dailySL, np.roll(belet_dictDP['Z0']['Center']['DP'],-147), lw = 0.5, label = 'Z0')
plt.legend()
plt.show()

In [ ]:
plt.plot(dailySL, np.roll(belet_dictDP['FL']['Center']['DP'],-147), lw = 0.5, label = 'FL', color = 'tab:blue')
plt.plot(dailySL, np.roll(belet_dictDP['WT']['Center']['DP'],-147), lw = 1, label = 'WT', color = 'tab:orange',alpha=0.5)
#plt.plot(dailySL, np.roll(belet_dictDP['WTZ0']['Center']['DP'],-147), lw = 0.5, label = 'WTZ0')
plt.legend()
plt.show()

## RDD Histograms

In [ ]:
count = 0
total = len(runs) * len(regions) * len(locations)

for run in runs:
    for region in regions:
        for location in locations:
            if region == 'Equator':
                location = ""
                timestep = "Timestep"
            else:
                timestep = ""

            name = "{}_{}_{}_".format(run,region,location)

            RDD_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYyrAvgTSt_RDD.csv')[0]

            RDD = openCSV(RDD_path)

            RDD = flattenArray(RDD)

            print("Current: ", RDD_path)

            output = "/Users/maxcollins/Desktop/organized images/Histograms/"+ name + 'RDD'

            histPlot(RDD, xlabel = 'Resultant Drift Direction (°)', bins=np.arange(0,361,22.5), output = output)

            count+=1

            ppercent(count,total)
            
            plt.clf()
                

## RDD Timeseries

In [ ]:
fensal_dictRDD = {}
belet_dictRDD = {}
shangrila_dictRDD = {}
eq_dictRDD = {}

region_dictsRDD = [belet_dictRDD, eq_dictRDD, fensal_dictRDD, shangrila_dictRDD]

count = 0
total = len(runs) * len(regions)

for index, region in enumerate(regions):
    for run in runs:
        
        if region == 'Equator':
            locations=''
        else:
            locations = ['Center', 'EastEdge']
            
        makeDict(region_dictsRDD[index], run, region, locations=locations, fn='*MYyrAvgTSt_RDD')
    
        count+=1
    
        ppercent(count,total)

In [ ]:
from numpy import diff
dx = dailySL[1] - dailySL[0]
y = belet_dictRDD['WT']['Center']
dy = diff(y)/dx
# plt.plot(dailySL[1:], np.roll(dy,rollbackDay))
yhat1 = savgol_filter(dy, 25, 5) # window size 51, polynomial order 3
plt.plot(dailySL[1:], np.roll(yhat1,rollbackDay))

In [ ]:
total = len(region_dict) * len(runs) * len(locations)
count=0

for index, region_dict in enumerate(region_dicts):
    for run in runs:
        for location in locations:           
            try:
                y =  region_dict[run][location]
                name = "{}_{}_{}_".format(run,region_names[index],location)
            except TypeError:
                y =  region_dict[run]
                name = "{}_{}_".format(run,region_names[index])
            
            output = "/Users/maxcollins/Desktop/organized images/Timeseries/RDD/"+region_names[index]+'/'+ name + 'RDD'
            
            yticks = np.arange(0,361,30)
            ylim = np.array([0,360])
            
            seriesPlot(y,'Resultant Drift Direction (°)','daily', output = output, yticks = yticks, ylim = ylim)
            
            plt.clf()
            count+=1
            
            ppercent(count,total)

#### RDD Comp Series

In [ ]:
# region_dictsRDD = [belet_dictRDD, eq_dictRDD, fensal_dictRDD, shangrila_dictRDD]
# region_names = ['Belet', 'Equator', 'Fensal', 'ShangriLa']
# locations = ['Center', 'EastEdge']
# runs = ['FL', 'WT', 'Z0', 'WTZ0']

comps = [('FL','WT'), ('FL','Z0'), ('WTZ0','Z0')]

In [ ]:
plt.clf()

count=0
total= len(region_dictsRDD) * len(locations) * len(comps)

for index, region_dict in enumerate(region_dictsRDD):
    for location in locations: 
        for comp in comps:
           
            if region_names[index] != 'Equator':
                
                name = "{}{}_{}_{}_RDD".format(comp[0],comp[1],region_names[index],location)
                
                RDD1 = region_dict[comp[0]][location]
                RDD2 = region_dict[comp[1]][location]
                
                RDD1_ERR = [sem(region_dict[comp[0]][location])]*675
                RDD2_ERR = [sem(region_dict[comp[1]][location])]*675

            else:
                
                name = name = "{}{}_{}_RDD".format(comp[0],comp[1],region_names[index])
                
                RDD1 = region_dict[comp[0]]
                RDD2 = region_dict[comp[1]]
                
                RDD1_ERR = [sem(region_dict[comp[0]])]*675
                RDD2_ERR = [sem(region_dict[comp[1]])]*675

            output = '/Users/maxcollins/Desktop/organized images/Timeseries/RDD/Comparison/' + name

            compSeries(RDD1, RDD2, [comp[0], comp[0]+' Standard Error'], [comp[1], comp[1]+' Standard Error'], 
                       'Solar Longitude (°)', 'Resultant Drift Direction (°)' , err1 = np.array(RDD1_ERR), 
                       err2 = np.array(RDD2_ERR), xticks = np.arange(0,361,30), yticks = np.arange(0,361,30), 
                       ylim = np.array([0.,360.]), output = output)
            count+=1
            ppercent(count,total)



In [ ]:
RDD1 = belet_dictRDD['FL']['Center']
RDD2 = belet_dictRDD['WT']['Center']

compSeries(RDD1, RDD2, ['Smoothed FL', 'FL'], ['Smoothed WT', 'WT'], 
           'Solar Longitude (°)', 'Resultant Drift Direction (°)' , xticks = np.arange(0,361,30),
           yticks = np.arange(0,361,30))

In [ ]:
RDD1 = belet_dictRDD['FL']['EastEdge']
RDD2 = belet_dictRDD['WT']['EastEdge']

compSeries(RDD1, RDD2, ['Smoothed FL', 'FL'], ['Smoothed WT', 'WT'], 
           'Solar Longitude (°)', 'Resultant Drift Direction (°)' , xticks = np.arange(0,361,30),
           yticks = np.arange(0,361,30))

In [ ]:
RDD1 = belet_dictRDD['FL']['EastEdge']
RDD2 = belet_dictRDD['Z0']['EastEdge']

compSeries(RDD1, RDD2, ['Smoothed FL', 'FL'], ['Smoothed Z0', 'Z0'], 
           'Solar Longitude (°)', 'Resultant Drift Direction (°)' , xticks = np.arange(0,361,30),
           yticks = np.arange(0,361,30))

In [ ]:
RDD1 = belet_dictRDD['WT']['EastEdge']
RDD2 = belet_dictRDD['WTZ0']['EastEdge']

compSeries(RDD1, RDD2, ['Smoothed WT', 'WT'], ['Smoothed WTZ0', 'WTZ0'], 
           'Solar Longitude (°)', 'Resultant Drift Direction (°)' , xticks = np.arange(0,361,30),
           yticks = np.arange(0,361,30))

## Zonal Wind Timeseries

In [ ]:
# UT = 0.005*24.3

count = 0
total = len(runs) * len(regions) * len(locations)

for run in runs:
    for region in regions:
        for location in locations:
            if region == 'Equator':
                location = ""
                timestep = "Timestep"
            else:
                timestep = ""

            name = "{}_{}_{}_".format(run,region,location)

            U_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYYrAvgTSt_U.csv')[0]
            
            U = openCSV(U_path)

            U = np.array(flattenArray(U))

            print("Current: ", U_path)

            output = "/Users/maxcollins/Desktop/organized images/Timeseries/"+ name + 'U'
            
            seriesPlot(U,'Wind Speed (m/s)','timestep', average=True, output = output)
            
            count+=1

            perc=float(count/total)*100

            text = "Percent completed: {:.2f}% ".format(perc)

            print(text)

            clear_output(wait=True)

## Meridional Wind Timeseries

In [ ]:
# UT = 0.005*24.3

count = 0
total = len(runs) * len(regions) * len(locations)

for run in runs:
    for region in regions:
        for location in locations:
            if region == 'Equator':
                location = ""
                timestep = "Timestep"
            else:
                timestep = ""

            name = "{}_{}_{}_".format(run,region,location)

            V_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYyrAvgTSt_V.csv')[0]
            
            V = openCSV(V_path)

            V = np.array(flattenArray(V))

            print("Current: ", V_path)

            output = "/Users/maxcollins/Desktop/organized images/Timeseries/"+ name + 'V'
            
            seriesPlot(U,'Wind Speed (m/s)','timestep', average=True, output = output)
            
            count+=1

            perc=float(count/total)*100

            text = "Percent completed: {:.2f}% ".format(perc)

            print(text)

            clear_output(wait=True)

## Wind Direction Timeseries

In [ ]:
# UT = 0.005*24.3

count = 0
total = len(runs) * len(regions) * len(locations)

for run in runs:
    for region in regions:
        for location in locations:
            if region == 'Equator':
                location = ""
                timestep = "Timestep"
            else:
                timestep = ""

            name = "{}_{}_{}_".format(run,region,location)

            DIR_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYYrAvgTSt_Dir.csv')[0]
            
            DIR = openCSV(DIR_path)

            DIR = np.array(flattenArray(DIR))

            print("Current: ", DIR_path)

            output = "/Users/maxcollins/Desktop/organized images/Timeseries/"+ name + 'DIR'
            
            seriesPlot(DIR,'Wind Direction (°)','timestep', average=True, output = output)
            
            count+=1

            perc=float(count/total)*100

            text = "Percent completed: {:.2f}% ".format(perc)

            print(text)

            clear_output(wait=True)

## Wind Magnitude Timeseries

In [ ]:
# UT = 0.005*24.3

count = 0
total = len(runs) * len(regions) * len(locations)

for run in runs:
    for region in regions:
        for location in locations:
            if region == 'Equator':
                location = ""
                timestep = "Timestep"
            else:
                timestep = ""

            name = "{}_{}_{}_".format(run,region,location)

            W_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYYrAvgTSt_W.csv')[0]
            
            W = openCSV(W_path)

            W = np.array(flattenArray(W))

            print("Current: ", W_path)

            output = "/Users/maxcollins/Desktop/organized images/Timeseries/"+ name + 'W'
            
            seriesPlot(W,'Wind Speed (m/s)','timestep', average=True, output = output)
            
            count+=1

            perc=float(count/total)*100

            text = "Percent completed: {:.2f}% ".format(perc)

            print(text)

            clear_output(wait=True)

## Frequency Days above Threshold
### and when those days occur during the year i.e. if during equinox

In [ ]:
UF = np.array([0.005, 0.01, 0.02])
UT = UF*24.3

count = 0
total = len(runs) * len(regions) * len(locations) * len(UT)

for run in runs:
    for region in regions:
        for location in locations:
            for ut in UT:
                
                if region == 'Equator':
                    location = ""
                    timestep = "Timestep"
                    name = "{}_{}_{}_".format(run,region,np.round(ut,2))
                else:
                    timestep = ""
                    name = "{}_{}_{}_{}_".format(run,region,location,np.round(ut,2))

                W_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYYrAvgTSt_W.csv')[0]
                DIR_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYYrAvgTSt_Dir.csv')[0]

                W = openCSV(W_path)

                DIR = openCSV(DIR_path)

                print("Current: ", W_path)
                print("Current: ", DIR_path)

                output = "/Users/maxcollins/Desktop/organized images/Histograms/"+ name

                plotWindFreq(W, DIR, ut, output = output)
                
                count+=1

                ppercent(count,total)


## Gusts

In [ ]:
count = 0
total = len(runs) * len(regions) * len(locations)

for run in runs:
    for region in regions:
        for location in locations:

            if region == 'Equator':
                location = ""
                timestep = "Timestep"
                name = "{}_{}_gust".format(run,region)
            else:
                timestep = ""
                name = "{}_{}_{}_gust".format(run,region,location)

            U_path = glob.glob('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+region+'/'+timestep+location+'/*MYYrAvgTSt_U.csv')[0]
            
            U = openCSV(U_path)
            
            print("Current: ", U_path)
        
            output = "/Users/maxcollins/Desktop/organized images/Scatter/"+ name
            
            wmax, emax = zonalGusts(U)
            
            plotScatter(emax, wmax, output = output)

            count+=1

            ppercent(count,total)

## Box plot comparisons

#### RDD

In [ ]:
yticks = np.arange(0,361,30)

for index, region_dict in enumerate(region_dictsRDD):

    for location in locations:
        
        if region_names[index]=='Equator':
            location = 'Equator'
            name = location + '_RDD'

        else:
            name = region_names[index] + location + '_RDD'

        
        output = '/Users/maxcollins/Desktop/organized images/Boxplots/' + name
        
        makeBoxPlot(region_dict, runs, location, 'Run', 'Resultant Drift Direction (°)', 0.0, 360., 
                    output = output, yticks = yticks)

#### DP

In [ ]:
#yticks = np.arange(0,361,30)

for index, region_dict in enumerate(region_dictsDP):

    for location in locations:

        plt.clf()
        
        if region_names[index]=='Equator':
            location = 'Equator'
            name = location + '_DP'

        else:
            name = region_names[index] + location + '_DP'

        
        output = '/Users/maxcollins/Desktop/organized images/Boxplots/' + name
        
        makeBoxPlot(region_dict, runs, location, 'Run', 'Drift Potential (VU)', 0.0, 0.015, 
                    output = output, var = 'DP')

## Friction Speed Velocity?

In [ ]:
beletUSTAR = {}
fensalUSTAR = {}
shangrilaUSTAR = {}
eqUSTAR = {}

region_dictsUSTAR = [beletUSTAR, eqUSTAR, fensalUSTAR, shangrilaUSTAR]

count = 0
total = len(runs) * len(regions)

for index, region in enumerate(regions):
    for run in runs:
        
        if region == 'Equator':
            locations=''
        else:
            locations = ['Center', 'EastEdge']
            
        makeDict(region_dictsUSTAR[index], run, region, locations=locations, USTAR = True)
    
        count+=1
    
        ppercent(count,total)

In [ ]:
eqUSTAR['FL']

In [ ]:
plt.clf()
sns.set_style("whitegrid")
plt.rcParams.update({'font.size': 16})
plt.figure(figsize=(8,6))

gfg = sns.histplot(eqUSTAR['FL'], bins=20, 
             common_norm = True, common_bins=True, stat = 'frequency', edgecolor='white', kde=True) # FL first

plt.xlabel("Friction Speed")

plt.savefig('/Users/maxcollins/Desktop/organized images/Histograms/Friction Speed/FL_USTAR.png', bbox_inches='tight')

plt.show()

In [ ]:
def ustarHistComp(compdict, run1, run2, location, hue_order, output):
    run = [run1]*len(compdict[run1][location])
    run.extend(([run2]*len(compdict[run2][location])))
    
    ustar = np.concatenate((compdict[run1][location], compdict[run2][location]))
    
    d_ustar = {'Run': run, 'Friction Speed': ustar}
    df_ustar = pd.DataFrame(data=d_ustar)
    
    plt.clf()
    sns.set_style("whitegrid")
    plt.rcParams.update({'font.size': 16})
    plt.figure(figsize=(8,6))

    gfg = sns.histplot(data=df_ustar, x="Friction Speed", hue="Run", multiple="layer", bins=20, 
                 common_norm = True, common_bins=True, stat = 'frequency', edgecolor='white', hue_order=hue_order) # FL first
    
    plt.setp(gfg.get_legend().get_title(), fontsize='20') 

    plt.savefig('/Users/maxcollins/Desktop/organized images/Histograms/Friction Speed/'+output+'_USTAR.png', bbox_inches='tight')

    plt.show()

In [ ]:
ustarHistComp(fensalUSTAR, 'FL', 'Z0', 'EastEdge', ['FL','Z0'], 'fenEE_FLZ0')

In [ ]:
run = ['WT']*len(beletUSTAR['WT']['Center'])
run.extend((['FL']*len(beletUSTAR['FL']['Center'])))

In [ ]:
ustar = np.concatenate((beletUSTAR['Z0']['Center'], beletUSTAR['FL']['Center'])) 

In [ ]:
d_ustar = {'Run': run, 'Friction Speed': ustar}
df_ustar = pd.DataFrame(data=d_ustar)


In [ ]:
plt.clf()
sns.set_style("whitegrid")
plt.rcParams.update({'font.size': 16})
plt.figure(figsize=(8,6))

gfg = sns.histplot(data=df_ustar, x="Friction Speed", hue="Run", multiple="layer", bins=20, 
             common_norm = True, common_bins=True, stat = 'frequency', edgecolor='white', hue_order=["FL", "Z0"]) 
# for legend text
#plt.setp(gfg.get_legend().get_texts(), fontsize='18') 
 
# for legend title
plt.setp(gfg.get_legend().get_title(), fontsize='20') 

plt.savefig('/Users/maxcollins/Desktop/organized images/Histograms/Friction Speed/belC_FLZ0_USTAR.png', bbox_inches='tight')

plt.show()

In [ ]:
# - Sand drift potential depends strongly on latitude and threshold friction - calculations with different grain size etc?
# - Latitudinal distribution of observed dunes implies threshold friction speed close to 0.02 ms-1 (Lorenz et al. 2006)
# - Allison (1992) threshold frictions speed 0.017 m s-1  for optimum sand diameter of 50 microns
# - Lorenz et al. 1995 min threshold friction speed on present Titan 0.03-0.04 ms-1 for 200-300 micron diameter

# The sand particle density is assumed to be 1000 kg m −3(Lorenz et al.,  1995), 0.02 m s−1 is most likely, 
# in accordance with Lorenz et al. (2006) . Allison (1992) calculated  the  threshold  friction  speed  as  0.017  m s − 1 for  
# an optimum sand diameter of 50 μm, while Lorenz et al. (1995) obtained a minimum threshold friction speed 
# on present Titan as 0.03–0.04 m s−1for 200–300 μm diameter

# Remember using threshold speed thats friction speed * 24.3 - also model output approximated closer to ground

# - Likely particle diameter 100 to 300 microns (Lorenz et al. 2006)
# - Threshold suspension fraction speed (Lorenz et al. 1995) varies between 0.08 and 0.3 ms-1
# - Beyond tail of wind probability density function, neglect likelihood of dune destruction by excessive winds

# - Basic factors that control formation of dunes are availability of sand, surface wind speed statistics, modality of wind direction (e.g.,Bagnold, 1941; Livingstone and Warren, 1996).
# - Speeds high enough for saltation but low enough for dune formation 


# * Eq. (13), grain size influences roughness because grains are the discrete units from which the bed is composed and larger grains 
# produce rougher microtopography and hence greater drag on the boundary layer whether or not saltation is occurring.
# * During saltation (i.e. when u⁎ − u⁎t> 0), an additional “saltation-induced” roughness occurs, represented by the second 
# term on the right side of Eq. (15). The magnitude of this effect increases with the excess shear velocity, because higher values 
# of excess shear velocity cause greater saltation. 
# https://click.endnote.com/viewer?doi=10.1016/j.geomorph.2008.10.010&route=5
# * The aerodynamic “law of the wall ”states that the wind velocity profile above a rough surface is a function of the aerodynamic roughness length, which, during sal-tation, is a function of both grain size and excess shear velocity (Sherman, 1992)
# * In areas of limited sand supply (i.e. partial sand cover), the aerodynamic roughness length will be at least partially controlled by the roughness elements (e.g. vegetation, alluvial deposits, and bedrock outcrops) that comprise the interdune or sand-free areas
# * the aerodynamic roughness length will become progressively more influenced by the size of the roughness elements in the interdune areas (which generally have no relationship with the grain size of sand on the dunes)
# J. D. Pelletier
# * increase in bed shear stress is associated with the fact that, for relatively small roughness lengths, the wind velocities interacting with the bedforms will be greater (due to the higher velocity gradients associated with smaller roughness lengths), hence the degree of flow compression/expansion will also be greater
# * At smaller roughness lengths (0.01 mm), the shear stress variation decreases, associated with increase in form drag and back pressure that limits near-surface wind velocities compared to cases with lower L/z0 ratios (L = half length of bedform)
# Geomorphology
# (2009)


## Vectors

In [ ]:
import cartopy.feature as cfeat
import matplotlib.pyplot as plt

import iris
import iris.plot as iplt
import iris.quickplot as qplt
from iris.coords import DimCoord
from iris.cube import Cube

In [ ]:
topography = '/Users/maxcollins/Documents/Code/python/Input Files/test files/titan_topo.nc'

In [ ]:
ds = xr.open_dataset(topography)

In [ ]:
run='WTZ0'

In [ ]:
u = openCSV('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+run+'AIJu_grid.csv')
v = openCSV('/Users/maxcollins/Documents/Code/python/data/'+run+'/'+run+'AIJv_grid.csv')

In [ ]:
latitude = DimCoord(np.arange(-90., 91., 4),
                    standard_name='latitude',
                    units='degrees')
longitude = DimCoord(np.arange(-177.5, 177.6, 5),
                     standard_name='longitude',
                     units='degrees')
uwind = Cube(u, units='m/s',
            dim_coords_and_dims=[(latitude, 0),
                                 (longitude, 1)])

uwind.rename('x_wind')

In [ ]:
latitude = DimCoord(np.arange(-90., 91., 4),
                    standard_name='latitude',
                    units='degrees')
longitude = DimCoord(np.arange(-177.5, 177.6, 5),
                     standard_name='longitude',
                     units='degrees')
vwind = Cube(v, units='m/s',
            dim_coords_and_dims=[(latitude, 0),
                                 (longitude, 1)])

vwind.rename('y_wind')

In [ ]:
latitude = DimCoord(np.arange(-90., 91., 4),
                    standard_name='latitude',
                    units='degrees')
longitude = DimCoord(np.arange(-177.5, 177.6, 5),
                     standard_name='longitude',
                     units='degrees')
topog = Cube(np.array(ds.zatmo), units='m',
            dim_coords_and_dims=[(latitude, 0),
                                 (longitude, 1)])

In [ ]:
plt.clf()
name = run+'_Vec'
#plt.figure(figsize=(15,15), constrained_layout=True)

fig, ax = plt.subplots(figsize=(15,15), constrained_layout=True)

X, Y = np.meshgrid(lats,lons)

# Create a cube containing the wind speed.
windspeed = (uwind ** 2 + vwind ** 2) ** 0.5
windspeed.rename("windspeed")

# Plot the wind speed as a contour plot.
if run != 'FL' and run != 'Z0':
    levels=[-1000, -800, -600, -400,-200, 0, 100, 150]
    qplt.contour(topog, colors='black', linewidths = 1.5, levels = levels, alpha=0.5)

levels = np.arange(0.,1.61,0.2)
qplt.contourf(windspeed, 20, cmap = 'hot_r', alpha=0.6, levels=levels)

# Add arrows to show the wind vectors.

plt.xlim([-180,180])
plt.xticks([-180,-150,-120,-90,-60,-30,0,30,60,90,120,150,180])

plt.yticks([-90,-60,-30,0,30,60,90])

plt.title(" ")

plt.ylabel('Latitude (°N)')
plt.xlabel('Longitude (°E)')
plt.axis([-180,180,-60,60])


plt.colorbar(orientation='vertical',pad=0.02,fraction=0.02, label='Wind Speed (m/s)', ticks=np.arange(0.,1.61,0.2))

iplt.quiver(uwind, vwind, X, Y, pivot="middle", scale=25, width=0.0015)

plt.savefig('/Users/maxcollins/Desktop/organized images/Vectors/'+name+'.png', bbox_inches='tight')

qplt.show()

### T-Tests

In [ ]:
from scipy import stats

t, p = ttest_ind(fensal_dictRDD['FL']['Center'], fensal_dictRDD['WT']['Center'])

In [ ]:
def statsDict(var_dict, run1, location = '', var = ''):
    
    dict = {}
    
    runs = ['FL', 'WT', 'Z0', 'WTZ0']
    
    for run in runs:
        
        if not location:
            if var:
                t, p = ttest_ind(var_dict[run1][var], var_dict[run][var])
            else:
                t, p = ttest_ind(var_dict[run1], var_dict[run])
        else:
            if var:
                t, p = ttest_ind(var_dict[run1][location][var], var_dict[run][location][var])
            else:
                t, p = ttest_ind(var_dict[run1][location], var_dict[run][location])
        
        dict[run] = (t, p)
    
    return dict

In [ ]:
# region_dictsDP
# region_dictsRDD = [belet_dictRDD, eq_dictRDD, fensal_dictRDD, shangrila_dictRDD]
stats_RDD = {}
stats_DP = {}

In [ ]:
for index, run in enumerate(runs):
    loc_dict = {}
    for location in locations:
        if index != 1:
            loc_dict[location] = statsDict(region_dictsRDD[index], run, location)
        else:
            loc_dict[location] = statsDict(region_dictsRDD[index], run)
            
    stats_RDD[run] = loc_dict

In [ ]:
for run in runs: 
    print(ttest_ind(belet_dictRDD['WT']['Center'], belet_dictRDD[run]['Center']))
    
print() 
    
for run in runs: 
    print(ttest_ind(belet_dictRDD['WT']['EastEdge'], belet_dictRDD[run]['EastEdge']))






In [ ]:
for index, run in enumerate(runs):
    
    loc_dict = {}
    
    for location in locations:
        if index != 1:
            loc_dict[location] = statsDict(region_dictsDP[index], run, location, 'DP')
        else:
            loc_dict[location] = statsDict(region_dictsDP[index], run, 'DP')
            
    stats_DP[run] = loc_dict